<a href="https://colab.research.google.com/github/afortuny/DeepLearningProduction/blob/main/Image_extractor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
#hide 
!pip install -Uqq fastbook # remember to restart runtime every time you run this line
import fastbook
fastbook.setup_book()

Mounted at /content/gdrive


In [7]:
#hide
from fastbook import *
from fastai.vision.widgets import *

In [15]:
from bs4 import BeautifulSoup
import os
import requests
import urllib
import base64
import re

In [10]:
search_words = ['adidas running shoe', 'adidas train shoe', 'grey shoes', 'nike running shoes', 'shoes']

In [11]:
img_dir = '/content/gdrive/MyDrive/Deep Learning Production/Images'

In [13]:
for word in search_words:
  dir_path = img_dir + word
  if not os.path.exists(dir_path):
    os.makedirs(dir_path)

In [17]:
urlKeyword = urllib.parse.quote(word)

In [19]:
url = 'https://www.google.com/search?hl=jp&q=' + urlKeyword + '&btnG=Google+Search&tbs=0&safe=off&tbm=isch'

In [20]:
headers = {"User-Agent": "Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:47.0) Gecko/20100101 Firefox/47.0",}

In [21]:
r=requests.get(url, headers=headers)

In [22]:
soup = BeautifulSoup(r.content, 'html.parser')

from bs4 import BeautifulSoup
import os
import requests
import urllib
import base64
import re